In [ ]:
import pymongo
cli = pymongo.MongoClient()
db = cli.test
coll = db.restaurants

In [ ]:
coll.find_one()

# B-Tree Indexes

In [ ]:
coll.drop_indexes()

In [ ]:
coll.index_information()

In [ ]:
coll.create_index('name')

In [ ]:
coll.create_index([('name', 1)])

In [ ]:
coll.create_index([('name', pymongo.ASCENDING)])

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
q.count()

In [ ]:
set(doc['cuisine'] for doc in q)

In [ ]:
coll.create_index([('name', 1), ('cuisine', 1)])

In [ ]:
coll.drop_index('name_1')

In [ ]:
q = coll.find(
    {'name': 'Dunkin Donuts', 'cuisine': 'Armenian'},
    {'name': 1, 'borough': 1, '_id': 0})
list(q)    

# Geospatial queries and indexing

In [ ]:
coll.create_index([
    ('address.coord', '2dsphere')  # longitude, latitude 
])

In [ ]:
doc = coll.find_one()
doc

In [ ]:
q = coll.find(
    {'address.coord': {
        '$nearSphere': doc['address']['coord']
    }},
    {'_id': 0, 'name': 1, 'address.coord': 1}
).limit(5)
list(q)

In [ ]:
point = {
    '$type': 'Point',
    'coordinates': doc['address']['coord']
}
q = coll.find(
    {'address.coord': {'$nearSphere': {'$geometry': point}}},
    {'_id': 0, 'name': 1, 'address.coord': 1}
).limit(5)
list(q)

In [ ]:
res = db.command('geoNear', 'restaurants', near=point, spherical=True)
res

In [ ]:
for i, doc in enumerate(res['results']):
    if i > 5:
        break
    print('{:.1f} meters away: {}'.format(doc['dis'], doc['obj']['name']))

# Full-text search and indexing

In [ ]:
import re
q_re = coll.find({'name': re.compile('^Dunkin')})
q_re.count()

In [ ]:
coll.create_index([('name', 'text')])

In [ ]:
q_text = coll.find({'$text': {
    '$search': 'Dunkin',
    '$caseSensitive': True
}})
q_text.count()

In [ ]:
found_with_re = {doc['_id'] for doc in q_re}
found_with_text = {doc['_id'] for doc in q_text}

In [ ]:
print('Found with re but not text:')
for _id in found_with_re - found_with_text:
    print(coll.find_one({'_id': _id}, {'name': 1}))
print('Found with text but not re:')
for _id in found_with_text - found_with_re:
    print(coll.find_one({'_id': _id}, {'name': 1}))

In [ ]:
coll.drop_indexes()

In [ ]:
coll.create_index([("$**", "text")])

In [ ]:
q_text = coll.find({'$text': {
    '$search': 'donut'     # finds everything with donut in name, cuisine, borough, or address
}})
q_text.count()

In [ ]:
q_text = coll.find({'$text': {
    '$search': 'queen'     # finds everything with queen in name, cuisine, borough, or address (incl. "Queens")
}})
q_text.count()

## Sorting by score

In [ ]:
coll.drop_indexes()
coll.create_index([('name', 'text')])

In [ ]:
q_text = coll.find(
    {'$text': {'$search': 'dunkin'}},
    {'score': {'$meta': 'textScore'}}
)

In [ ]:
q_text.next()

In [ ]:
q_text = coll.find(
    {'$text': {'$search': 'dunkin'}},
    {'score': {'$meta': 'textScore'}}
)
q_text = q_text.sort([('score', {'$meta': 'textScore'})])

In [ ]:
q_text.next()